In [1]:
# Cellule 1 — Imports & Paths

import os
from pathlib import Path
import pandas as pd
import numpy as np
import cv2
from tqdm import tqdm

# Dossiers de travail
BASE_DIR = Path("/home/fabrice/Boneage")

CSV_PATH = BASE_DIR / "boneage-training-dataset-cleaned-v3.csv"
IMG_DIR = BASE_DIR / "boneage-training-dataset" / "boneage-training-dataset"
MASK_DIR = BASE_DIR / "eff_unet"

OUTPUT_DIR = BASE_DIR / "boneage_segmented_eff_unet"
OUTPUT_DIR.mkdir(exist_ok=True)

print("CSV :", CSV_PATH)
print("Images :", IMG_DIR)
print("Masques :", MASK_DIR)
print("Output :", OUTPUT_DIR)


CSV : /home/fabrice/Boneage/boneage-training-dataset-cleaned-v3.csv
Images : /home/fabrice/Boneage/boneage-training-dataset/boneage-training-dataset
Masques : /home/fabrice/Boneage/eff_unet
Output : /home/fabrice/Boneage/boneage_segmented_eff_unet


In [2]:
# Cellule 2 — Charger le CSV

df = pd.read_csv(CSV_PATH)

print("Nombre d'images dans le CSV :", len(df))
df.head()


Nombre d'images dans le CSV : 11325


,id,boneage,male
0,1377,180,False
1,1380,120,True
2,1382,138,True
3,1385,36,True
4,1387,138,True


In [3]:
# Cellule 3 — Fonctions utilitaires

def load_image(path):
    """Charge une image en niveaux de gris."""
    return cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)

def load_mask(path):
    """Charge un masque Eff-UNet et le binarise entre 0 et 1."""
    mask = cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)
    if mask is None:
        return None
    _, mask_bin = cv2.threshold(mask, 1, 1, cv2.THRESH_BINARY)
    return mask_bin

def apply_mask(image, mask):
    """Applique le masque : multiplication pixel par pixel."""
    return image * mask


In [4]:
# Cellule 4 — Boucle de segmentation complète

missing_masks = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    img_id = str(row["id"])
    filename = f"{img_id}.png"
    
    img_path = IMG_DIR / filename
    mask_path = MASK_DIR / filename
    
    # Charger image
    image = load_image(img_path)
    if image is None:
        print(f" Image introuvable : {filename}")
        continue
    
    # Charger masque
    mask = load_mask(mask_path)
    if mask is None:
        missing_masks.append(filename)
        continue
    
    # Appliquer le masque
    masked_image = apply_mask(image, mask)
    
    # Sauvegarder
    save_path = OUTPUT_DIR / filename
    cv2.imwrite(str(save_path), masked_image)

print("Segmentation terminée.")


100%|█████████████████████████████████████████████████████████████████████████████| 11325/11325 [09:26<00:00, 19.98it/s]

Segmentation terminée.
